In [ ]:
tg_list =['https://t.me/mobile_jobs']
api_id = '21227109'  
api_hash = ''

In [6]:



import asyncio
import pandas as pd
import time
from telethon.sync import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from urllib.parse import urlparse
import random
import nest_asyncio
nest_asyncio.apply()

# Конфигурация
max_messages = 500
data_dir = "data"
session_name = "job_parser_session"


# Функция получения только message и date
async def fetch_message_and_date(client, channel_url):
    username = urlparse(channel_url).path.strip('/')
    try:
        entity = await client.get_entity(username)
    except Exception as e:
        print(f"❌ Не удалось получить {username}: {e}")
        return

    all_rows = []
    offset_id = 0
    seen_messages = set()

    while True:
        try:
            history = await client(GetHistoryRequest(
                peer=entity,
                offset_id=offset_id,
                offset_date=None,
                add_offset=0,
                limit=100,
                max_id=0,
                min_id=0,
                hash=0
            ))
        except Exception as e:
            print(f"⚠️ Ошибка получения истории: {e}")
            break

        if not history.messages:
            break

        for msg in history.messages:
            try:
                text = msg.message
                date = msg.date.isoformat() if hasattr(msg, 'date') else None
                
                if text:
                    norm_text = text.strip().lower()
                    # опционально под канал mobile_jobs
                    if '#резюме' in norm_text or '#ищу' in norm_text:
                        continue
                    if norm_text in seen_messages:
                        continue
                    if '#вакансия' not in norm_text:
                        continue

                    seen_messages.add(norm_text)
                    all_rows.append({
                        "message": text.strip(), 
                        "date": date,
                        "url": f"https://t.me/{username}/{msg.id}"
                    })
            except Exception as e:
                print(f"⚠️ Пропущено сообщение из-за ошибки: {e}")

        offset_id = history.messages[-1].id
        
        if len(all_rows) >= max_messages:
            break


        await asyncio.sleep(random.uniform(5, 15))

    df = pd.DataFrame(all_rows)
    df.to_csv(f"{data_dir}/{username}.csv", index=False, encoding='utf-8-sig')
    print(f"✅ Сохранено: {username}.csv c {len(df)} сообщений")


# Основной запуск парсера
async def main():

    async with TelegramClient(session_name, api_id, api_hash) as client:
        for url in tg_list:
            try:
                print(f"\n📥 Обработка: {url}")
                await fetch_message_and_date(client, url)
                await asyncio.sleep(random.uniform(5, 10))
            except Exception as e:
                print(f"❌ Ошибка с {url}: {e}")
                await asyncio.sleep(5)


await main()

Signed in successfully as Andrey Shveduke; remember to not break the ToS or you will risk an account ban!

📥 Обработка: https://t.me/it_match_ml_ds
⚠️ Ошибка получения истории: The key is not registered in the system (caused by GetHistoryRequest)
✅ Сохранено: it_match_ml_ds.csv c 3 сообщений


In [9]:
tg_list =['https://t.me/it_match_ml_ds']

In [ ]:
import asyncio
import pandas as pd
from telethon import TelegramClient
from telethon.errors import FloodWaitError
from urllib.parse import urlparse
import random
import os


max_messages = 400
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

async def fetch_message_and_date(client, channel_url):
    username = urlparse(channel_url).path.strip('/')
    try:
        entity = await client.get_entity(username)
    except Exception as e:
        print(f"❌ Не удалось получить {username}: {e}")
        return

    all_rows = []
    seen_messages = set()

    try:
        async for msg in client.iter_messages(entity, limit=max_messages):
            if not msg.message:
                continue
            norm_text = msg.message.strip().lower()

            if '#резюме' in norm_text or '#ищу' in norm_text:
                continue
            if norm_text in seen_messages:
                continue
            if '#вакансия' not in norm_text:
                continue

            seen_messages.add(norm_text)
            all_rows.append({
                "message": msg.message.strip(),
                "date": msg.date.isoformat(),
                "url": f"https://t.me/{username}/{msg.id}"
            })

            await asyncio.sleep(random.uniform(0.5, 2))  # мини-пауза между сообщениями

    except FloodWaitError as e:
        print(f"⏳ FloodWait: жду {e.seconds} сек")
        await asyncio.sleep(e.seconds + 5)
    except Exception as e:
        print(f"⚠️ Ошибка получения: {e}")

    if all_rows:
        df = pd.DataFrame(all_rows)
        df.to_csv(f"{data_dir}/{username}.csv", index=False, encoding='utf-8-sig')
        print(f"✅ Сохранено: {username}.csv c {len(df)} сообщений")


async def main():
    async with TelegramClient("job_parser_session", api_id, api_hash) as client:
        for url in tg_list:
            print(f"\n📥 Обработка: {url}")
            await fetch_message_and_date(client, url)
            await asyncio.sleep(random.uniform(20, 60))  # большая пауза между каналами

asyncio.run(main())

Signed in successfully as Andrey Shveduke; remember to not break the ToS or you will risk an account ban!

📥 Обработка: https://t.me/mobile_jobs
✅ Сохранено: mobile_jobs.csv c 26 сообщений


In [ ]:

tg_list =['https://t.me/mobile_jobs']
api_id = '21227109'  
api_hash = ''
        

In [ ]:
import asyncio
import json
from telethon import TelegramClient
from telethon.errors.rpcerrorlist import FloodWaitError

API_ID = 21227109
API_HASH = ''
PHONE = '+79250454426'

CHANNEL = 'https://t.me/hireproproduct/'
OUTPUT_FILE = 'hireproproduct.json'
LIMIT = 1500  # количество сообщений для скачивания

client = TelegramClient('session_name', API_ID, API_HASH)

async def main():
    
    await client.start(phone=PHONE)

    all_messages = []
    try:
        async for message in client.iter_messages(CHANNEL, limit=LIMIT):
            msg_data = {
                'id': message.id,
                'date': message.date.isoformat(),
                'text': message.text,
                'sender_id': message.sender_id,
                'media_type': type(message.media).__name__ if message.media else None,
            }
            all_messages.append(msg_data)

    except FloodWaitError as e:
        print(f'Flood wait: нужно подождать {e.seconds} секунд.')
    except Exception as e:
        print(f'Ошибка: {e}')

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(all_messages, f, ensure_ascii=False, indent=2)

    print(f'Всего сообщений сохранено: {len(all_messages)}')


if __name__ == '__main__':
    asyncio.run(main())


c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\telethon\tl\types\__init__.py:22893: RuntimeWarning: coroutine 'main' was never awaited
  self.offset = offset


Всего сообщений сохранено: 948


Cannot get difference since the account is likely misusing the session: database is locked
Unhandled exception from keepalive_handle after cancelling <class 'asyncio.tasks.Task'> (<Task finished name='Task-170' coro=<UpdateMethods._keepalive_loop() done, defined at c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\telethon\client\updates.py:481> exception=OperationalError('database is locked')>)
Traceback (most recent call last):
  File "c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\telethon\helpers.py", line 176, in _cancel
    await task
  File "c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\asyncio\futures.py", line 287, in __await__
    return self.result()  # May raise too.
  File "c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\asyncio\futures.py", line 201, in result
    raise self._exception
  File "c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\asyncio\tasks.py", line 256, in __step
    r

In [ ]:
# ПОПЫТКА запустить сохранение каанлов серией 

import asyncio
import json
from telethon import TelegramClient
from telethon.errors.rpcerrorlist import FloodWaitError

API_ID = 21227109
API_HASH = 'da2cdcb70a47e04b23b1320970726e0c'
PHONE = '+79250454426'

CHANNELS = [
    'https://t.me/analyst_job',
    'https://t.me/hireproproduct',
    'https://t.me/Remoteit',
    'https://t.me/devops_jobs_feed/',
    'https://t.me/Getitrussia/',
    'https://t.me/fordev'
    
]

LIMIT = 1000
DELAY_BETWEEN_CHANNELS = 10

async def download_channel_messages(client, channel_url):
    output_file = f"{channel_url.split('/')[-1]}_messages.json"
    print(f"Начинаю скачивание сообщений из канала {channel_url}...")
    
    all_messages = []
    try:
        async for message in client.iter_messages(channel_url, limit=LIMIT):
            msg_data = {
                'id': message.id,
                'date': message.date.isoformat(),
                'text': message.text,
                'sender_id': message.sender_id,
                'media_type': type(message.media).__name__ if message.media else None,
            }
            all_messages.append(msg_data)

    except FloodWaitError as e:
        print(f'Flood wait: нужно подождать {e.seconds} секунд.')
        await asyncio.sleep(e.seconds)
        return await download_channel_messages(client, channel_url)
    except Exception as e:
        print(f'Ошибка при скачивании {channel_url}: {e}')
        return None

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_messages, f, ensure_ascii=False, indent=2)

    print(f'Сохранено {len(all_messages)} сообщений из {channel_url} в файл {output_file}')
    return output_file

async def main():
    client = TelegramClient('session_name', API_ID, API_HASH)
    await client.start(phone=PHONE)
    
    downloaded_files = []
    for channel in CHANNELS:
        try:
            result = await download_channel_messages(client, channel)
            if result:
                downloaded_files.append(result)
            
            if channel != CHANNELS[-1]:
                print(f"Ожидание {DELAY_BETWEEN_CHANNELS} секунд перед следующим каналом...")
                await asyncio.sleep(DELAY_BETWEEN_CHANNELS)
                
        except Exception as e:
            print(f'Критическая ошибка при обработке канала {channel}: {e}')
    
    await client.disconnect()
    print("\nГотово! Скачанные файлы:")
    for file in downloaded_files:
        print(f"- {file}")

# Универсальный способ запуска
if __name__ == '__main__':
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # Для Jupyter/интерактивных сред
            import nest_asyncio
            nest_asyncio.apply()
            loop.run_until_complete(main())
        else:
            # Для обычного запуска
            asyncio.run(main())
    except RuntimeError as e:
        print(f"Ошибка: {e}")

Signed in successfully as Andrey Shveduke; remember to not break the ToS or you will risk an account ban!
Начинаю скачивание сообщений из канала https://t.me/analyst_job...
Сохранено 1000 сообщений из https://t.me/analyst_job в файл analyst_job_messages.json
Ожидание 10 секунд перед следующим каналом...
Начинаю скачивание сообщений из канала https://t.me/hireproproduct...
Ошибка при скачивании https://t.me/hireproproduct: The key is not registered in the system (caused by ResolveUsernameRequest)
Ожидание 10 секунд перед следующим каналом...
Начинаю скачивание сообщений из канала https://t.me/Remoteit...
Ошибка при скачивании https://t.me/Remoteit: The key is not registered in the system (caused by ResolveUsernameRequest)
Ожидание 10 секунд перед следующим каналом...
Начинаю скачивание сообщений из канала https://t.me/devops_jobs_feed/...
Ошибка при скачивании https://t.me/devops_jobs_feed/: The key is not registered in the system (caused by ResolveUsernameRequest)
Ожидание 10 секунд пе

In [ ]:
# пересохраняем в csv

import json
import csv
from pathlib import Path
from datetime import datetime

def json_to_csv(json_file_path, csv_file_path):
    # Чтение JSON файла
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        messages = json.load(json_file)
    
    # Подготовка данных для CSV
    csv_data = []
    for message in messages:
        # Форматирование даты (можно оставить как есть или преобразовать)
        date = message['date']
        # Можно преобразовать дату в другой формат, например:
        # date_obj = datetime.fromisoformat(message['date'])
        # date = date_obj.strftime('%Y-%m-%d %H:%M:%S')
        
        csv_data.append({
            'text': message['text'],
            'date': date
        })
    
    # Запись в CSV файл
    with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['text', 'date'])
        writer.writeheader()
        writer.writerows(csv_data)

def process_all_json_files(input_folder, output_folder):
    input_path = Path(input_folder)
    output_path = Path(output_folder)
    
    # Создаем папку для выходных файлов, если ее нет
    output_path.mkdir(exist_ok=True)
    
    # Обрабатываем все JSON файлы в папке
    for json_file in input_path.glob('*.json'):
        csv_file = output_path / f"{json_file.stem}.csv"
        json_to_csv(json_file, csv_file)
        print(f"Converted {json_file.name} to {csv_file.name}")




In [15]:
input_folder = "json_tg_2/"  
output_folder = "csv_tg/"  
    
process_all_json_files(input_folder, output_folder)

Converted analyst_job_messages.json to analyst_job_messages.csv
Converted devops_jobs_feed.json to devops_jobs_feed.csv
Converted fordev_messages.json to fordev_messages.csv
Converted Getitrussia.json to Getitrussia.csv
Converted hireproproduct.json to hireproproduct.csv


In [ ]:
# отдельно для remoteit

import asyncio
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from telethon import TelegramClient
from telethon.errors.rpcerrorlist import FloodWaitError

# --- Настройки Telegram ---
API_ID = 21227109
API_HASH = ''
PHONE = '+79250454426'
CHANNEL = 'https://t.me/Remoteit'
LIMIT = 500  # Кол-во сообщений

# --- Настройки сохранения ---
OUTPUT_FILE = "remoteit_teletype.csv"

# Регулярка для поиска ссылки teletype
TELETYPE_PATTERN = re.compile(r"https?://teletype\.in/[^\s]+")

def extract_teletype_content(url):
    """Скачивает страницу teletype и извлекает текст статьи"""
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"[Ошибка запроса] {url} — {e}")
        return None

    soup = BeautifulSoup(resp.text, "html.parser")
    article = soup.find("article")
    if not article:
        return None
    
    text = " ".join(article.stripped_strings)
    return text

async def main():
    client = TelegramClient('session_name', API_ID, API_HASH)
    await client.start(phone=PHONE)

    results = []

    try:
        async for message in client.iter_messages(CHANNEL, limit=LIMIT):
            if not message.text:
                continue

            match = TELETYPE_PATTERN.search(message.text)
            if match:
                url = match.group(0)
                print(f"[+] {message.id} — {url}")
                content = extract_teletype_content(url)
                if content:
                    results.append({
                        "id": message.id,
                        "date": message.date.isoformat(),
                        "teletype_url": url,
                        "content": content
                    })

    except FloodWaitError as e:
        print(f"[FloodWait] Нужно подождать {e.seconds} секунд.")
    except Exception as e:
        print(f"[Ошибка] {e}")
    finally:
        await client.disconnect()

    # Сохраняем сразу в CSV
    df = pd.DataFrame(results)
    df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    print(f"[✓] Сохранено {len(results)} записей в {OUTPUT_FILE}")

if __name__ == "__main__":
    asyncio.run(main())


Task was destroyed but it is pending!
task: <Task pending name='Task-348' coro=<Connection._send_loop() running at c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\telethon\network\connection\connection.py:322> wait_for=<Future pending cb=[Task.__wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-349' coro=<Connection._recv_loop() running at c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\telethon\network\connection\connection.py:341> wait_for=<Future pending cb=[Task.__wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-350' coro=<MTProtoSender._send_loop() running at c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\telethon\network\mtprotosender.py:464> wait_for=<Future pending cb=[Task.__wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-351' coro=<MTProtoSender._recv_loop() running at c:\Users\Aero-PC\AppData\Local\Prog

OperationalError: database is locked

Server closed the connection: 0 bytes read on a total of 8 expected bytes
Error executing high-level request after reconnect: <class 'sqlite3.OperationalError'>: database is locked
Server closed the connection: 0 bytes read on a total of 8 expected bytes
Error executing high-level request after reconnect: <class 'sqlite3.OperationalError'>: database is locked
Server closed the connection: 0 bytes read on a total of 8 expected bytes
Error executing high-level request after reconnect: <class 'sqlite3.OperationalError'>: database is locked
Server closed the connection: 0 bytes read on a total of 8 expected bytes
Error executing high-level request after reconnect: <class 'sqlite3.OperationalError'>: database is locked
Server closed the connection: 0 bytes read on a total of 8 expected bytes
Error executing high-level request after reconnect: <class 'sqlite3.OperationalError'>: database is locked
Server closed the connection: 0 bytes read on a total of 8 expected bytes
Error executing high-